In [ ]:
import meso_tools as mt
import os
import numpy as np
import matplotlib.pyplot as pl
import tifffile
import sciris as sc

In [ ]:
four_roi_stack= 'E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\617911_surface_soma_4ROIS_00001.tif'

if not os.path.isfile(four_roi_stack):
print(f"!!!! 4 ROI stack was deleted! restore it before proceeding!!!!")

In [ ]:
#1. open tiff file
tiff=tifffile.TiffFile(four_roi_stack, mode='rb')
num_frames = len(tiff.pages)

In [ ]:
#stack parameters
ROIs = 4
image_dim = 768
spacer = 24

chunk_size = 100
chunk_num = int(np.ceil(num_frames / chunk_size))
# allocation of output vector
average_fluorescence = np.zeros((num_frames,))

curr_frame = 0
print(f"Stack will be processed in {chunk_num} chunks")

In [ ]:
# 2. Initiallize arrays for averaged stacks, mean and variance outputs
V1 = np.zeros((int(num_frames/chunk_size), image_dim , image_dim ))
LM = np.zeros((int(num_frames/chunk_size), image_dim , image_dim ))
PM = np.zeros((int(num_frames/chunk_size), image_dim , image_dim ))
AM = np.zeros((int(num_frames/chunk_size), image_dim , image_dim ))

mean = np.zeros((ROIs, int(num_frames/chunk_size)))
var = np.zeros((ROIs, int(num_frames/chunk_size)))

In [ ]:
# 3. loop over chunks to generate averaged stack and calculate mean and variance

for i in range(chunk_num):

    # load chunk of tiff:
    frame_start = chunk_size*i
    frame_end = chunk_size*(i+1)

    # handling of the last chunk 
    if frame_end > num_frames:
        frame_end = num_frames
    print(f'Loading {i}th chunk, frames {frame_start} to {frame_end}')
    tiff_array = tiff.asarray(range(frame_start,frame_end))

    num_frames_chunk = tiff_array.shape[0]

    # split and average 4 rois - single planes
    V1[i,:,:] = tiff_array[:,:image_dim,:].mean(axis=0)
    LM[i,:,:] = tiff_array[:,image_dim+spacer:2*image_dim+spacer,:].mean(axis=0)
    AM[i,:,:] = tiff_array[:,2*image_dim+2*spacer:3*image_dim+2*spacer,:].mean(axis=0)
    PM[i,:,:] = tiff_array[:,3*image_dim+3*spacer:,:].mean(axis=0)
    
    mean[0,i] = np.mean(V1[i,:,:],axis=(0,1))
    mean[1,i] = np.mean(LM[i,:,:],axis=(0,1))
    mean[2,i] = np.mean(AM[i,:,:],axis=(0,1))   
    mean[3,i] = np.mean(PM[i,:,:],axis=(0,1))
    var[0,i] = np.var(V1[i,:,:])
    var[1,i] = np.var(LM[i,:,:])        
    var[2,i] = np.var(AM[i,:,:])        
    var[3,i] = np.var(PM[i,:,:])
    
tiff.close()

In [ ]:
# 4. write dict with mean and avg data to disk
four_roi_data = {}
four_roi_data['roi0_mean'] = mean[0]
four_roi_data['roi1_mean'] = mean[1]
four_roi_data['roi2_mean'] = mean[2]
four_roi_data['roi3_mean'] = mean[3]

four_roi_data['roi0_var'] = var[0]
four_roi_data['roi1_var'] = var[1]
four_roi_data['roi2_var'] = var[2]
four_roi_data['roi3_var'] = var[3]

path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\4_roi_data.dict"
sc.saveobj(path, four_roi_data)

In [ ]:
# 5. write tiffs of individual averaged frames to disk

v1_path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\V1_avg.tiff"
lm_path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\LM_avg.tiff"
am_path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\AM_avg.tiff"
pm_path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\PM_avg.tiff"

mt.write_tiff(v1_path, V1.astype(np.int16))
mt.write_tiff(lm_path, LM.astype(np.int16))
mt.write_tiff(am_path, AM.astype(np.int16))
mt.write_tiff(pm_path, PM.astype(np.int16))